<h1>Dataset Preprocessing to Enable Application of Classical ML Models</h1>

In [1]:
import json

In [2]:
import numpy as np
import tensorflow as tf

In [3]:
from model.model import build_preprocessing

In [4]:
# Load configuration file from json in the given folder
with open("config.json", "r") as config_file:
    config = json.load(config_file)

In [5]:
# Set up the list of gestures
with open(config["Paths"]["Gesture list"], "r") as gesture_list:
    gestures = gesture_list.readlines()[0].split(", ")

In [6]:
img_size = config["General parameters"]["Image size"]

In [7]:
train_images, test_images = tf.keras.preprocessing.image_dataset_from_directory("Data",
                                                                                labels="inferred",
                                                                                label_mode="int",
                                                                                class_names=gestures,
                                                                                color_mode="rgb",
                                                                                batch_size=128,
                                                                                image_size=(img_size,
                                                                                            img_size),
                                                                                shuffle=True,
                                                                                seed=42,
                                                                                validation_split=0.2,
                                                                                subset="both")

Found 26351 files belonging to 49 classes.
Using 21081 files for training.
Using 5270 files for validation.


In [8]:
# Set the default preprocessing pipeline if not specified
preprocessing_layers = config["Model"]["Default preprocessing"]
preprocessing_layers = "I,G,R"

# Build the preprocessing pipeline according to given instructions
img_size = 64
resize = tf.keras.layers.Resizing(img_size, img_size)
preprocessing = build_preprocessing(inp_shape=[img_size,
                                               img_size,
                                               3],
                                    instructions=preprocessing_layers,
                                    name="preprocessing_pipeline")

In [9]:
# Apply the same preprocessing steps as for the CNN but flatten the images
channels = 1 if "G" in preprocessing_layers else 3
train_images = train_images.map(lambda x, y: (tf.reshape(preprocessing(resize(x)), [-1, img_size ** 2 * channels]), y)).unbatch()
test_images = test_images.map(lambda x, y: (tf.reshape(preprocessing(resize(x)), [-1, img_size ** 2 * channels]), y)).unbatch()

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [10]:
# Convert the tf.data.Datasets to numpy arrays containing image & label per sample
train_images = np.array(list(train_images.as_numpy_iterator()), dtype=tuple)
test_images = np.array(list(test_images.as_numpy_iterator()), dtype=tuple)

In [11]:
# Extract the images and the labels separately
train_X, train_y = np.array([element[0] for element in train_images]), np.ravel(np.vstack([element[1] for element in train_images]))
test_X, test_y = np.array([element[0] for element in test_images]), np.ravel(np.vstack([element[1] for element in test_images]))

In [12]:
# Check memory consumption for the training dataset
print("Training dataset memory consumption: {:.1f}MB".format(train_X.nbytes / (1024 * 1024.0)))

Training dataset memory consumption: 3089.3MB


<h1>k-NN Gesture Classification</h1>

In [13]:
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV

In [14]:
# Initialize and train the k-NN classifier with parameters obtained from grid search
knn = KNeighborsClassifier(n_neighbors=1,
                           p=1,
                           n_jobs=-1)
knn.fit(train_X, train_y)

KNeighborsClassifier(n_jobs=-1, n_neighbors=1, p=1)

In [15]:
# Evaluate the classifier on the test dataset
knn_predictions = knn.predict(test_X)

print("Accuracy:", accuracy_score(test_y, knn_predictions))
#print(classification_report(test_y,
#                            knn_predictions,
#                            labels=gestures[:-1],
#                            target_names=gestures[:-1]))

Accuracy: 0.9865275142314991


In [16]:
np.unique(knn_predictions, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]),
 array([104, 110, 117, 115,  94, 114, 115, 111, 103, 116,  93, 113,  95,
         98, 104, 121, 109, 126, 108, 110, 123, 116,  97, 123, 113, 107,
        102, 117,  88, 117, 117, 110, 102, 121, 111, 107, 115, 103,  99,
        110, 107, 117, 114, 128, 100,  93, 126, 111], dtype=int64))

In [17]:
np.unique(test_y, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]),
 array([104, 110, 114, 116,  93, 115, 115, 112, 100, 116,  92, 115,  95,
         98, 104, 122, 109, 126, 110, 110, 123, 114,  97, 123, 108, 107,
        102, 116,  91, 118, 119, 110,  99, 122, 113, 109, 115, 104,  98,
        110, 106, 110, 117, 125, 102,  98, 125, 113], dtype=int64))